In [46]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import datetime
import re
import json 
from pandas.api.types import is_numeric_dtype


In [47]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [48]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [49]:
resultadoAmsa = sf.query_all(
    """
        SELECT CALENDAR_YEAR(CreatedDate), COUNT(Id),CALENDAR_MONTH(CreatedDate)
    from Paciente_Programas__c
    Where Programas_por_Empresas__r.Holding__c like '%antofagasta minerals%'
    GROUP BY CALENDAR_YEAR(CreatedDate), CALENDAR_MONTH(CreatedDate)
    ORDER BY CALENDAR_MONTH(CreatedDate)
        """
)
amsaIngreso = pd.DataFrame(resultadoAmsa["records"])

amsaIngreso = amsaIngreso[
    [
        "expr0",
        "expr2",
        "expr1",
    ]
]
amsaIngreso = amsaIngreso.rename(
    columns={
        "expr0": "Año",
        "expr1": "Cantidad",
        "expr2": "Mes",
    }
)

meses = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}

amsaIngreso.replace({"Mes": meses}, inplace=True)

amsaIngreso.to_excel("F:/Reportes SF/Zurich/Cantida_AMSA.xlsx", index=False)
amsaIngreso.head(12)

,Año,Mes,Cantidad
0,2023,Enero,4086
1,2023,Febrero,267
2,2023,Marzo,195
3,2023,Mayo,3150
4,2023,Junio,346
5,2023,Julio,125
6,2023,Agosto,1
7,2023,Septiembre,174
8,2023,Octubre,200
9,2022,Noviembre,3


In [50]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false  
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(14314, 8)

In [51]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7769, 5)

In [52]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

(14320, 10)

In [53]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

PermissionError: [Errno 13] Permission denied: 'F:/Empresas SF/reportes/empresas.xlsx'

fin empresas y comienzo de rescate de pacientes


In [ ]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True 
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1886342, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


In [ ]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1502464, 4)

In [ ]:
# Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

(1502464, 13)

In [ ]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [ ]:
# pacientes_empresas22 = pd.merge(
# left=Pacientes_SF,
# right=empresas_unidas,
# how="left",
# left_on="Id Account Paciente",
# right_on="Id Account",
# )
# pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
# pacientes_empresas22 = pacientes_empresas22[
# ["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
# ]

In [ ]:
# pacientes_empresas22.shape

In [ ]:
# apprix_1 = pacientes_empresas22.iloc[:848826,:]

# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)

In [ ]:
# apprix_2 = pacientes_empresas22.iloc[848827:,:]#
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [ ]:
# pacientes_empresas22.head()

Carga de Archivo carga


In [ ]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "BCI Seguros Colectivos"
Pais = "Chile"
# Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion

In [ ]:
Pacientes_Archivo.head(10)

,PÓLIZA,,Holding,EMPLEADOR,NOMBRE_GRUPO,RUT_TITULAR,rut2,RUT_ASEGURADO,DV_ASEGURADO,rut1,NOMBRE DEL ASEGURADO,apellidos,nombres,RELACIÓN,sdfsdfsd
0,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,TRAFIGURA CHILE LTDA,12695049.0,12695049-7,12695049,7,126950497,ABDON FERNANDO SUAREZ ZAMORANO,SUAREZ ZAMORANO,ABDON FERNANDO,Titular Masculino,Plan Gold
1,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,TRAFIGURA CHILE LTDA,7779737.0,7779737-8,22041113,3,220411133,AGUSTINA SIBISA RIVADENEIRA,SIBISA RIVADENEIRA,AGUSTINA,Hija,Plan Gold
2,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,IMPALA TERMINALS CHILE SPA,25844901.0,25844901-0,25844901,0,258449010,ALBENIS JOSE CASTRO VARGAS,CASTRO VARGAS,ALBENIS JOSE,Titular Masculino,Plan Gold
3,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,C. STEINWEG HANDELSVEEM BV LTDA.,24926804.0,24926804-6,24926804,6,249268046,ALLAN SANTOS SAMPAIO,SANTOS SAMPAIO,ALLAN,Titular Masculino,Plan Gold
4,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,IMPALA TERMINALS CHILE SPA,16288694.0,16288694-0,24000979,K,24000979K,AMALIA PAZ CHOQUE HILL,CHOQUE HILL,AMALIA PAZ,Hijastra,Plan Gold
5,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,IMPALA TERMINALS CHILE SPA,15885934.0,15885934-3,25915266,6,259152666,AMBAR DOMINIQUE ZAMORA HUERTA,ZAMORA HUERTA,AMBAR DOMINIQUE,Hija,Plan Gold
6,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,C. STEINWEG HANDELSVEEM BV LTDA.,8547622.0,8547622-K,8547622,K,8547622K,AMERICO CERDA VERGARA,CERDA VERGARA,AMERICO,Titular Masculino,Plan Gold
7,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,IMPALA TERMINALS CHILE SPA,18235559.0,18235559-3,19970038,3,199700383,Ana Belén Zúñiga Barraza,Zúñiga Barraza,Ana Belén,Esposa,Plan Gold
8,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,IMPALA TERMINALS CHILE SPA,11518921.0,11518921-2,11933511,6,119335116,ANA MARIA LARA ROJAS,LARA ROJAS,ANA MARIA,Esposa,Plan Gold
9,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,IMPALA TERMINALS CHILE SPA,16288694.0,16288694-0,16288694,0,162886940,ANDRES EDUARDO FIGUEROA ROJO,FIGUEROA ROJO,ANDRES EDUARDO,Titular Masculino,Plan Gold


In [ ]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)

In [ ]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,PÓLIZA,,Holding,EMPLEADOR,NOMBRE_GRUPO,RUT_TITULAR,rut2,RUT_ASEGURADO,DV_ASEGURADO,rut1,NOMBRE DEL ASEGURADO,apellidos,nombres,RELACIÓN,sdfsdfsd,rut
0,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,TRAFIGURA CHILE LTDA,12695049.0,12695049-7,12695049,7,126950497,ABDON FERNANDO SUAREZ ZAMORANO,SUAREZ ZAMORANO,ABDON FERNANDO,Titular Masculino,Plan Gold,126950497
1,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,TRAFIGURA CHILE LTDA,7779737.0,7779737-8,22041113,3,220411133,AGUSTINA SIBISA RIVADENEIRA,SIBISA RIVADENEIRA,AGUSTINA,Hija,Plan Gold,220411133
2,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,IMPALA TERMINALS CHILE SPA,25844901.0,25844901-0,25844901,0,258449010,ALBENIS JOSE CASTRO VARGAS,CASTRO VARGAS,ALBENIS JOSE,Titular Masculino,Plan Gold,258449010
3,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,C. STEINWEG HANDELSVEEM BV LTDA.,24926804.0,24926804-6,24926804,6,249268046,ALLAN SANTOS SAMPAIO,SANTOS SAMPAIO,ALLAN,Titular Masculino,Plan Gold,249268046
4,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,IMPALA TERMINALS CHILE SPA,16288694.0,16288694-0,24000979,K,24000979K,AMALIA PAZ CHOQUE HILL,CHOQUE HILL,AMALIA PAZ,Hijastra,Plan Gold,24000979K


In [ ]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16808\1949753364.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,PÓLIZA,,Holding,EMPLEADOR,NOMBRE_GRUPO,RUT_TITULAR,rut2,RUT_ASEGURADO,DV_ASEGURADO,rut1,NOMBRE DEL ASEGURADO,apellidos,nombres,RELACIÓN,sdfsdfsd,rut
0,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,TRAFIGURA CHILE LTDA,12695049.0,12695049-7,12695049,7,126950497,ABDON FERNANDO SUAREZ ZAMORANO,SUAREZ ZAMORANO,ABDON FERNANDO,Titular Masculino,Plan Gold,126950497
1,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,TRAFIGURA CHILE LTDA,7779737.0,7779737-8,22041113,3,220411133,AGUSTINA SIBISA RIVADENEIRA,SIBISA RIVADENEIRA,AGUSTINA,Hija,Plan Gold,220411133
2,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,IMPALA TERMINALS CHILE SPA,25844901.0,25844901-0,25844901,0,258449010,ALBENIS JOSE CASTRO VARGAS,CASTRO VARGAS,ALBENIS JOSE,Titular Masculino,Plan Gold,258449010
3,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,C. STEINWEG HANDELSVEEM BV LTDA.,24926804.0,24926804-6,24926804,6,249268046,ALLAN SANTOS SAMPAIO,SANTOS SAMPAIO,ALLAN,Titular Masculino,Plan Gold,249268046
4,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,IMPALA TERMINALS CHILE SPA,16288694.0,16288694-0,24000979,K,24000979K,AMALIA PAZ CHOQUE HILL,CHOQUE HILL,AMALIA PAZ,Hijastra,Plan Gold,24000979K


In [ ]:
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rut2"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("-", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.upper()

Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.strip()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16808\1368669443.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")


In [ ]:
def siPrimerCero(y):
    if y == "00":
        return "Nones"
    if y[0] == "0":
        y = y[1:]
    return y

In [ ]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].apply(lambda y: siPrimerCero(y))

In [ ]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].apply(lambda y: siPrimerCero(y))

Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16808\620742920.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16808\620742920.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [ ]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(30873, 27)

In [ ]:
Pacientes_con_id.head()

,PÓLIZA,,Holding,EMPLEADOR,NOMBRE_GRUPO,RUT_TITULAR,rut2,RUT_ASEGURADO,DV_ASEGURADO,rut1,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,TRAFIGURA CHILE LTDA,12695049.0,12695049-7,12695049,7,126950497,...,True,0018c00002bJgpfAAC,126950497,Abdon Fernando Suarez Zamorano,RUT,None,None,None,Otro,None
1,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,TRAFIGURA CHILE LTDA,7779737.0,7779737-8,22041113,3,220411133,...,True,0018c00002givkmAAA,220411133,Agustina Sibisa Rivadeneira,RUT,None,None,None,None,None
2,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,IMPALA TERMINALS CHILE SPA,25844901.0,25844901-0,25844901,0,258449010,...,True,0018c00002bJiWsAAK,258449010,Albenis Jose Castro Vargas,RUT,None,None,None,Otro,None
3,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,C. STEINWEG HANDELSVEEM BV LTDA.,24926804.0,24926804-6,24926804,6,249268046,...,True,0018c00002bJlOoAAK,249268046,Allan Santos Sampaio,RUT,None,None,None,Otro,None
4,209903,209903,TRAFIGURA,TRAFIGURA CHILE LTDA.,IMPALA TERMINALS CHILE SPA,16288694.0,16288694-0,24000979,K,24000979K,...,True,0018c00002givtQAAQ,24000979K,Amalia Paz Choque Hill,RUT,None,None,None,None,None


In [ ]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")


def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""

In [ ]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
""" if ("rutT" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Titulares"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    ) """
    
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = Pacientes_con_id["email"].astype(str).str.lower()
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(
        lambda y: isValid(y)
    )
    # Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "H": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "Prefiero no responder",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "M": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon": "Hombre",
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("/", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("null", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NULL", "")


Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c


In [ ]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña
""" cha_arc = datetime.now()
dia_hoy = str(fecha_arc.day)
mes_hoy = str(fecha_arc.month)
 """


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
# clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)
""" clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)
 """

' clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)\n '

In [ ]:
""" print(dia_hoy) """

' print(dia_hoy) '

In [ ]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
157080,a1Y8c00000C6V01EAF,0018c00002amPZPAA2,a1W8c0000083rhoEAA,BCI Seguros Colectivos,0018c00002axQgtAAE,BCI Seguros,Trafigura,BCI Seguros Colectivos,Trafigura Chile Ltda.,209903,None,EMP6118,A-9328
693979,a1Y8c00000CyLq0EAF,0018c00002mSmxuAAC,a1W8c0000083rhzEAA,BCI Seguros Colectivos,0018c00002axQgmAAE,BCI Seguros,Conaf,BCI Seguros Colectivos,Corporación Nacional Forestal,1298618,None,EMP6259,A-9339
696213,a1Y8c00000CySbREAV,0018c00002mTrrxAAC,a1W8c0000083rhsEAA,BCI Seguros Colectivos,0018c00002axQh1AAE,BCI Seguros,Porvenir,BCI Seguros Colectivos,Soc. Agrícola El Porvenir S.A.,241535,None,EMP6126,A-9332
696214,a1Y8c00000CySbSEAV,0018c00002mTrryAAC,a1W8c0000083rhsEAA,BCI Seguros Colectivos,0018c00002axQh1AAE,BCI Seguros,Porvenir,BCI Seguros Colectivos,Soc. Agrícola El Porvenir S.A.,241535,None,EMP6126,A-9332
696215,a1Y8c00000CySbTEAV,0018c00002mTrrzAAC,a1W8c0000083rhsEAA,BCI Seguros Colectivos,0018c00002axQh1AAE,BCI Seguros,Porvenir,BCI Seguros Colectivos,Soc. Agrícola El Porvenir S.A.,241535,None,EMP6126,A-9332


In [ ]:
campaña_seleccionada.shape

(36178, 13)

In [ ]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )


In [ ]:
if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )

In [ ]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [ ]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)

In [ ]:
nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
#campaña = "BCI Seguros Colectivos"
#Pais = "Chile"
# Pais = "México"

rutes = pd.read_excel(nuevosrutes)

In [ ]:
#derty = rutes.to_numpy().transpose().tolist()

In [ ]:
rutesi = pd.merge(
    left=rutes,
    right=Pacientes_SF,
    how="left",
    left_on="Rut",
    right_on="Rut",
)
rutesi.shape

(30873, 9)

In [ ]:
rutesi.head()

,Rut,Id Account Paciente,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,126950497,0018c00002bJgpfAAC,Abdon Fernando Suarez Zamorano,RUT,None,None,None,Otro,None
1,220411133,0018c00002givkmAAA,Agustina Sibisa Rivadeneira,RUT,None,None,None,None,None
2,258449010,0018c00002bJiWsAAK,Albenis Jose Castro Vargas,RUT,None,None,None,Otro,None
3,249268046,0018c00002bJlOoAAK,Allan Santos Sampaio,RUT,None,None,None,Otro,None
4,24000979K,0018c00002givtQAAQ,Amalia Paz Choque Hill,RUT,None,None,None,None,None


In [ ]:

rutesi = pd.merge(
    left=rutesi,
    right=empresas_unidas,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)

In [ ]:
rutesi.head()

,Rut,Id Account Paciente,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono,Id Pacientes_Programas__c,...,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,126950497,0018c00002bJgpfAAC,Abdon Fernando Suarez Zamorano,RUT,None,None,None,Otro,None,a1Y8c00000DywB8EAJ,...,BCI Seguros Colectivos,0018c00002axQgtAAE,BCI Seguros,Trafigura,BCI Seguros Colectivos,Trafigura Chile Ltda.,209903,None,EMP6118,A-9328
1,126950497,0018c00002bJgpfAAC,Abdon Fernando Suarez Zamorano,RUT,None,None,None,Otro,None,a1Y8c00000E5jn7EAB,...,BCI Seguros Colectivos,0018c00002axQgsAAE,BCI Seguros,Trafigura,BCI Seguros Colectivos,C. Steinweg Handelsveem BV Chile Ltda.,209903,None,EMP6117,A-9327
2,220411133,0018c00002givkmAAA,Agustina Sibisa Rivadeneira,RUT,None,None,None,None,None,a1Y8c00000DywB5EAJ,...,BCI Seguros Colectivos,0018c00002axQgtAAE,BCI Seguros,Trafigura,BCI Seguros Colectivos,Trafigura Chile Ltda.,209903,None,EMP6118,A-9328
3,220411133,0018c00002givkmAAA,Agustina Sibisa Rivadeneira,RUT,None,None,None,None,None,a1Y8c00000E5jn4EAB,...,BCI Seguros Colectivos,0018c00002axQgsAAE,BCI Seguros,Trafigura,BCI Seguros Colectivos,C. Steinweg Handelsveem BV Chile Ltda.,209903,None,EMP6117,A-9327
4,258449010,0018c00002bJiWsAAK,Albenis Jose Castro Vargas,RUT,None,None,None,Otro,None,a1Y8c00000DyuoQEAR,...,BCI Seguros Colectivos,0018c00002axQguAAE,BCI Seguros,Trafigura,BCI Seguros Colectivos,Impala Terminals Chile SPA,209903,None,EMP6119,A-9329


In [ ]:
rutesi.to_excel(
    "F:/Empresas SF/reportes/rutesi11.xlsx", index=False
)